## Cargar las librerias necesarias

In [1]:
import pandas as pd # type: ignore
import unicodedata
import re
import ast
from unidecode import unidecode

## Cargar todas las tablas necesarias

In [2]:
categorias = pd.read_csv(r'data_normalizada\categories_normalized.csv')
business = pd.read_csv(r'data_normalizada\business_normal.csv')
metadata = pd.read_csv(r'data_normalizada\metadata_normal.csv')
reviews_y = pd.read_csv(r'data_normalizada\reviews_normal.csv')
reviews_gm = pd.read_csv(r'data_normalizada\reviews_gm_normal.csv')
metadata_categories = pd.read_csv(r'data_normalizada\metadata_categories.csv')
business_categories = pd.read_csv(r'data_normalizada\business_categories.csv')
ciudades = pd.read_csv(r'data_normalizada\ciudades_normal.csv')

### Transformacion de `business` y `metadata`. Unificación en `negocios`

In [3]:
# Limpiar la columna 'id': quitar espacios y acentos
metadata['id'] = metadata['id'].apply(lambda x: unidecode(x).replace(" ", ""))
business['id'] = business['id'].apply(lambda x: unidecode(x).replace(" ", ""))

In [4]:
# Convertir 'category_id' en listas reales si es que aún son strings
metadata['category_id'] = metadata['category_id'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reemplazar listas vacías con [66] antes de explotar
metadata['category_id'] = metadata['category_id'].apply(lambda x: [66] if isinstance(x, list) and len(x) == 0 else x)

# Expandir la relación de muchos a muchos
metadata_intermedia = metadata[['id', 'category_id']].explode('category_id')

# Resetear el índice después de expandir
metadata_intermedia = metadata_intermedia.reset_index(drop=True)

# Verificar la estructura final
print(metadata_intermedia.info())
print(metadata_intermedia.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21829 entries, 0 to 21828
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           21829 non-null  object
 1   category_id  21829 non-null  object
dtypes: object(2)
memory usage: 341.2+ KB
None
                         id category_id
0  aagrillxxxxx261688801011          45
1  abacoagolfcl268885801199          61
2  abacusbusine284586814547          37
3  abbaletelavi257688801366          37
4  abbesdonutsh270467822499          37


In [5]:
# Convertir 'category_id' en listas reales si es que aún son strings
business['category_id'] = business['category_id'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reemplazar listas vacías con [66] antes de explotar
business['category_id'] = business['category_id'].apply(lambda x: [66] if isinstance(x, list) and len(x) == 0 else x)

# Expandir la relación de muchos a muchos
business_intermedia = business[['id', 'category_id']].explode('category_id')

# Resetear el índice después de expandir
business_intermedia = business_intermedia.reset_index(drop=True)

# Verificar la estructura final
print(business_intermedia.info())
print(business_intermedia.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14693 entries, 0 to 14692
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           14693 non-null  object
 1   category_id  14693 non-null  object
dtypes: object(2)
memory usage: 229.7+ KB
None
                         id category_id
0  aamorepizzer278603827290          26
1  aapizzaxxxxx283578826647          19
2  aapizzaxxxxx283578826647          26
3  aapizzaxxxxx283578826647          27
4  abbottstatio282351821827          37


In [6]:
# Concatenar las tablas
negocios_categorias = pd.concat([business_intermedia, metadata_intermedia], ignore_index=True)

# Verificar si hay registros duplicados en el par (id, category_id)
duplicados = negocios_categorias.duplicated(subset=['id', 'category_id'], keep=False)

# Eliminar duplicados dejando solo una aparición de cada par (id, category_id)
negocios_categorias = negocios_categorias.drop_duplicates(subset=['id', 'category_id'])

# Verificar la estructura final
print(negocios_categorias.info())
print(negocios_categorias.head())

# Guardar la tabla
negocios_categorias.to_csv(r'modelado/negocios_categorias.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 36326 entries, 0 to 36521
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           36326 non-null  object
 1   category_id  36326 non-null  object
dtypes: object(2)
memory usage: 851.4+ KB
None
                         id category_id
0  aamorepizzer278603827290          26
1  aapizzaxxxxx283578826647          19
2  aapizzaxxxxx283578826647          26
3  aapizzaxxxxx283578826647          27
4  abbottstatio282351821827          37


In [7]:
# Seleccionar las columnas binarias
columnas_atributos = [
    "delivery", "takeout", "dinein", "outdoor_seating", "drivethrough",
    "good_for_working_on_laptop", "solo_dining", "wheelchair_friendly",
    "alcohol_beverage", "healthy_food", "fast_comfort_food",
    "braille_menu", "all_you_can_eat", "coffee", "dancing",
    "catering", "counter_service", "pay_ahead", "seating",
    "dogs_allowed", "bike_parking", "parking",
    "breakfast", "lunch", "dinner", "dessert",
    "casual", "romantic", "formal", "trendy",
    "with_reservation", "usually_a_wait", "quick_visit",
    "black_owned", "women_led", "veteran_led",
    "entertainment", "live_entertainment", "lgbtq_friendly",
    "fast_service", "fireplace", "rooftop_seating", "sports",
    "college_students", "family_friendly", "groups",
    "locals", "tourists", "kids_friendly",
    "wifi", "bar_onsite", "cash_only", "checks", "credit_cards", 
    "debit_cards", "nfc_mobile_payments", "recycling", "none"
]

In [8]:
# Crear el DataFrame a partir de la lista
df_atributos = pd.DataFrame({
    "atributo_id": range(1, len(columnas_atributos) + 1),  # Generar IDs secuenciales
    "atributo": columnas_atributos
})

# Guardar el DataFrame de atributos normalizado
df_atributos.to_csv('modelado/atributos.csv', index=False)

In [9]:
metadata.rename(columns={"wi_fi": "wifi"}, inplace=True)

In [10]:
def generar_lista_atributos(df, columnas_atributos):
    """
    Convierte las columnas binarias en una lista de atributos presentes en cada negocio.
    """
    # Verificar que df no sea None
    if df is None:
        raise ValueError("El DataFrame proporcionado es None. Asegúrate de que está correctamente cargado.")

    # Filtrar solo las columnas que existen en el DataFrame actual
    columnas_validas = [col for col in columnas_atributos if col in df.columns]

    # Verificar que haya columnas válidas antes de aplicar la transformación
    if not columnas_validas:
        print("No hay columnas binarias en el DataFrame actual.")
        df = df.copy()
        df["atributos"] = [[]] * len(df)  # Crear una columna vacía para evitar errores
        return df

    # Copia el DataFrame para evitar problemas de SettingWithCopyWarning
    df = df.copy()

    # Reemplazar valores NaN con 0 y asegurarse de que los datos sean enteros
    df[columnas_validas] = df[columnas_validas].fillna(0).astype(int)

    # Aplicar la transformación a las columnas disponibles
    df["atributos"] = df[columnas_validas].apply(
        lambda row: [col for col in columnas_validas if row[col] == 1], axis=1
    )

    return df

# Verificar antes de aplicar la transformación
if metadata is None or business is None:
    raise ValueError("metadata o business están vacíos o no están correctamente definidos.")

# Aplicamos la transformación asegurándonos de que cada DataFrame solo usa sus columnas disponibles
metadata = generar_lista_atributos(metadata, columnas_atributos)
business = generar_lista_atributos(business, columnas_atributos)

# Verificar que la transformación se aplicó correctamente
print(metadata[['id', 'atributos']].head())
print(business[['id', 'atributos']].head())

                         id                                          atributos
0  aagrillxxxxx261688801011        [takeout, fast_comfort_food, kids_friendly]
1  abacoagolfcl268885801199                              [wheelchair_friendly]
2  abacusbusine284586814547                    [delivery, wheelchair_friendly]
3  abbaletelavi257688801366  [delivery, takeout, dinein, outdoor_seating, s...
4  abbesdonutsh270467822499  [delivery, takeout, dinein, solo_dining, wheel...
                         id                                          atributos
0  aamorepizzer278603827290  [delivery, takeout, outdoor_seating, wheelchai...
1  aapizzaxxxxx283578826647  [delivery, takeout, outdoor_seating, wheelchai...
2  abbottstatio282351821827  [takeout, outdoor_seating, catering, seating, ...
3  abbyshealthn280783825080  [takeout, wheelchair_friendly, catering, seati...
4  abcfinewinea280147826140  [delivery, seating, bike_parking, parking, cre...


In [11]:
# Si algún registro en metadata["atributos"] está vacío, asignarle ["none"]
metadata["atributos"] = metadata["atributos"].apply(lambda x: ["none"] if len(x) == 0 else x)

# Expandir la lista de atributos en metadata (cada fila tendrá un solo atributo)
df_intermedia = metadata[["id", "atributos"]].explode("atributos")

# Unir con df_atributos para obtener el atributo_id
df_intermedia = df_intermedia.merge(df_atributos, left_on="atributos", right_on="atributo", how="left")

# Seleccionar solo las columnas necesarias y renombrarlas
df_intermedia = df_intermedia[["id", "atributo_id"]]#.rename(columns={"id": "metadata_id"})

# Verificar si hubo atributos que no hicieron match
if df_intermedia["atributo_id"].isna().sum() > 0:
    print("Algunos atributos no tienen un ID asignado.")
    print(df_intermedia[df_intermedia["atributo_id"].isna()].head())

# Convertir atributo_id a int (si no hay NaN)
df_intermedia["atributo_id"] = df_intermedia["atributo_id"].astype("Int64")

print(df_intermedia.info())
print(df_intermedia.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150886 entries, 0 to 150885
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           150886 non-null  object
 1   atributo_id  150886 non-null  Int64 
dtypes: Int64(1), object(1)
memory usage: 2.4+ MB
None
                         id  atributo_id
0  aagrillxxxxx261688801011            2
1  aagrillxxxxx261688801011           11
2  aagrillxxxxx261688801011           49
3  abacoagolfcl268885801199            8
4  abacusbusine284586814547            1


In [12]:
# Si algún registro en business["atributos"] está vacío, asignarle ["none"]
business["atributos"] = business["atributos"].apply(lambda x: ["none"] if len(x) == 0 else x)

# Expandir la lista de atributos en business (cada fila tendrá un solo atributo)
df_intermedia_y = business[["id", "atributos"]].explode("atributos")

# Unir con df_atributos para obtener el atributo_id
df_intermedia_y = df_intermedia_y.merge(df_atributos, left_on="atributos", right_on="atributo", how="left")

# Seleccionar solo las columnas necesarias y renombrarlas
df_intermedia_y = df_intermedia_y[["id", "atributo_id"]]#.rename(columns={"id": "business_id"})

# Verificar si hubo atributos que no hicieron match
if df_intermedia_y["atributo_id"].isna().sum() > 0:
    print("Algunos atributos no tienen un ID asignado.")
    print(df_intermedia_y[df_intermedia_y["atributo_id"].isna()].head())

# Convertir atributo_id a int (si no hay NaN)
df_intermedia_y["atributo_id"] = df_intermedia_y["atributo_id"].astype("Int64")

print(df_intermedia_y.info())
print(df_intermedia_y.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100142 entries, 0 to 100141
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           100142 non-null  object
 1   atributo_id  100142 non-null  Int64 
dtypes: Int64(1), object(1)
memory usage: 1.6+ MB
None
                         id  atributo_id
0  aamorepizzer278603827290            1
1  aamorepizzer278603827290            2
2  aamorepizzer278603827290            4
3  aamorepizzer278603827290            8
4  aamorepizzer278603827290            9


In [13]:
# Concatenar las tablas
negocios_atributos = pd.concat([df_intermedia_y, df_intermedia], ignore_index=True)

# Verificar si hay registros duplicados en el par (id, atributo_id)
duplicados = negocios_atributos.duplicated(subset=['id', 'atributo_id'], keep=False)

# Eliminar duplicados dejando solo una aparición de cada par (id, atributo_id)
negocios_atributos = negocios_atributos.drop_duplicates(subset=['id', 'atributo_id'])

# Verificar la estructura final
print(negocios_atributos.info())
print(negocios_atributos.head())

# Guardar la tabla
negocios_atributos.to_csv(r'modelado/negocios_atributos.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 249747 entries, 0 to 251027
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           249747 non-null  object
 1   atributo_id  249747 non-null  Int64 
dtypes: Int64(1), object(1)
memory usage: 6.0+ MB
None
                         id  atributo_id
0  aamorepizzer278603827290            1
1  aamorepizzer278603827290            2
2  aamorepizzer278603827290            4
3  aamorepizzer278603827290            8
4  aamorepizzer278603827290            9


In [14]:
metadata_limpio = metadata.drop(columns = ['delivery', 'takeout', 'dinein','outdoor_seating', 
                                           'drivethrough', 'good_for_working_on_laptop', 'solo_dining', 
                                           'wheelchair_friendly', 'alcohol_beverage', 'healthy_food', 
                                           'fast_comfort_food', 'braille_menu', 'all_you_can_eat', 'coffee', 
                                           'dancing', 'catering', 'counter_service', 'pay_ahead','seating', 
                                           'breakfast', 'lunch', 'dinner', 'dessert', 'casual', 'romantic', 
                                           'formal', 'trendy', 'with_reservation', 'usually_a_wait', 
                                           'quick_visit', 'black_owned', 'women_led', 'veteran_led', 
                                           'entertainment', 'live_entertainment', 'lgbtq_friendly', 
                                           'fast_service', 'fireplace', 'rooftop_seating', 'sports', 
                                           'college_students', 'family_friendly', 'groups', 'locals', 
                                           'tourists', 'kids_friendly', 'wifi','bar_onsite', 'cash_only', 
                                           'checks', 'credit_cards', 'debit_cards', 'nfc_mobile_payments', 
                                           'recycling'])

print(metadata_limpio.info())
print(metadata_limpio.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15782 entries, 0 to 15781
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              15782 non-null  object 
 1   name            15782 non-null  object 
 2   street_address  15567 non-null  object 
 3   postal_code     15782 non-null  int64  
 4   latitude        15782 non-null  float64
 5   longitude       15782 non-null  float64
 6   city_id         15782 non-null  int64  
 7   category_id     15782 non-null  object 
 8   stars           15782 non-null  float64
 9   review_count    15782 non-null  int64  
 10  is_open         15782 non-null  int64  
 11  atributos       15782 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 1.4+ MB
None
                         id                       name  \
0  aagrillxxxxx261688801011                  A1A Grill   
1  abacoagolfcl268885801199           Abacoa Golf Club   
2  abacusbusine28458

In [15]:
# Convertir tipos de datos
metadata_limpio['id'] = metadata_limpio['id'].astype(str)
metadata_limpio['name'] = metadata_limpio['name'].astype(str)
metadata_limpio['street_address'] = metadata_limpio['street_address'].astype(str)

metadata_limpio['postal_code'] = metadata_limpio['postal_code'].astype('int64')
metadata_limpio['city_id'] = metadata_limpio['city_id'].astype('int64')
metadata_limpio['review_count'] = metadata_limpio['review_count'].astype('int64')
metadata_limpio['is_open'] = metadata_limpio['is_open'].astype('int64')

# Redondear latitud y longitud a 7 decimales
metadata_limpio['latitude'] = metadata_limpio['latitude'].round(7)
metadata_limpio['longitude'] = metadata_limpio['longitude'].round(7)

# Redondear estrellas a 1 decimal
metadata_limpio['stars'] = metadata_limpio['stars'].round(1)


In [16]:
business_limpio = business.drop (columns = ['delivery', 'takeout', 'outdoor_seating', 'drivethrough',
                                            'wheelchair_friendly', 'alcohol_beverage', 'dancing', 'catering',
                                            'counter_service', 'seating', 'dogs_allowed', 'bike_parking', 
                                            'parking', 'breakfast', 'lunch', 'dinner', 'dessert', 'casual', 
                                            'romantic', 'formal', 'trendy', 'with_reservation', 
                                            'live_entertainment', 'groups', 'kids_friendly', 'wifi', 
                                            'bar_onsite', 'credit_cards'])

print(business_limpio.info())
print(business_limpio.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10933 entries, 0 to 10932
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10933 non-null  object 
 1   name          10933 non-null  object 
 2   postal_code   10930 non-null  float64
 3   latitude      10933 non-null  float64
 4   longitude     10933 non-null  float64
 5   city_id       10933 non-null  int64  
 6   category_id   10933 non-null  object 
 7   stars         10933 non-null  float64
 8   review_count  10933 non-null  int64  
 9   address       10807 non-null  object 
 10  is_open       10933 non-null  int64  
 11  atributos     10933 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 1.0+ MB
None
                         id                            name  postal_code  \
0  aamorepizzer278603827290  A' Amor'e Pizzeria Grill & Pub      33782.0   
1  aapizzaxxxxx283578826647                     A & A Pizza      34667

In [17]:
# Convertir tipos de datos
business_limpio['id'] = business_limpio['id'].astype(str)
business_limpio['name'] = business_limpio['name'].astype(str)
business_limpio['address'] = business_limpio['address'].astype(str)

#business_limpio['postal_code'] = business_limpio['postal_code'].astype('int64')
business_limpio['city_id'] = business_limpio['city_id'].astype('int64')
business_limpio['review_count'] = business_limpio['review_count'].astype('int64')
business_limpio['is_open'] = business_limpio['is_open'].astype('int64')

# Redondear latitud y longitud a 7 decimales
business_limpio['latitude'] = business_limpio['latitude'].round(7)
business_limpio['longitude'] = business_limpio['longitude'].round(7)

# Redondear estrellas a 1 decimal
business_limpio['stars'] = business_limpio['stars'].round(1)

In [18]:
print(metadata_limpio.info())
print(business_limpio.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15782 entries, 0 to 15781
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              15782 non-null  object 
 1   name            15782 non-null  object 
 2   street_address  15782 non-null  object 
 3   postal_code     15782 non-null  int64  
 4   latitude        15782 non-null  float64
 5   longitude       15782 non-null  float64
 6   city_id         15782 non-null  int64  
 7   category_id     15782 non-null  object 
 8   stars           15782 non-null  float64
 9   review_count    15782 non-null  int64  
 10  is_open         15782 non-null  int64  
 11  atributos       15782 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 1.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10933 entries, 0 to 10932
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  

In [19]:
# Renombrar 'address' a 'street_address' en business_limpio
business_limpio = business_limpio.rename(columns={'address': 'street_address'})

# Convertir postal_code de business_limpio a int64, reemplazando NaN por 0
business_limpio['postal_code'] = business_limpio['postal_code'].fillna(0).astype('int64')

# Asegurar que ambas tablas tienen las mismas columnas y en el mismo orden
columnas_ordenadas = [
    'id', 'name', 'street_address', 'postal_code', 'latitude', 'longitude', 
    'city_id', 'category_id', 'stars', 'review_count', 'is_open', 'atributos'
]

metadata_limpio = metadata_limpio[columnas_ordenadas]
business_limpio = business_limpio[columnas_ordenadas]

# Concatenar las tablas
negocios = pd.concat([metadata_limpio, business_limpio], ignore_index=True)

# Verificar la estructura final
print(negocios.info())
print(negocios.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26715 entries, 0 to 26714
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26715 non-null  object 
 1   name            26715 non-null  object 
 2   street_address  26715 non-null  object 
 3   postal_code     26715 non-null  int64  
 4   latitude        26715 non-null  float64
 5   longitude       26715 non-null  float64
 6   city_id         26715 non-null  int64  
 7   category_id     26715 non-null  object 
 8   stars           26715 non-null  float64
 9   review_count    26715 non-null  int64  
 10  is_open         26715 non-null  int64  
 11  atributos       26715 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.4+ MB
None
                         id                       name  \
0  aagrillxxxxx261688801011                  A1A Grill   
1  abacoagolfcl268885801199           Abacoa Golf Club   
2  abacusbusine28458

In [20]:
negocios_limpio = negocios.drop(columns=['category_id','atributos'])

In [21]:
# Contar cuántas veces aparece cada ID
duplicados_id = negocios_limpio['id'].value_counts()

# Filtrar solo los IDs que aparecen más de una vez
duplicados_id = duplicados_id[duplicados_id > 1]

# Mostrar los duplicados si existen
if not duplicados_id.empty:
    print(f"Se encontraron {len(duplicados_id)} IDs duplicados en la tabla negocios.")
    print(duplicados_id)
else:
    print("No hay IDs duplicados en la tabla negocios.")

Se encontraron 390 IDs duplicados en la tabla negocios.
id
goldenwokxxx284910825404    2
littleleonxx279363825026    2
rockinwingsx280227827719    2
mcdonaldsxxx281878825463    2
subwayxxxxxx279391822546    2
                           ..
wokchistirfr281875823501    2
yborbeverage279593824368    2
yuppixxxxxxx279600824384    2
zzpizzasxxxx279862825700    2
adryanaslati280163821228    2
Name: count, Length: 390, dtype: int64


In [22]:
# Eliminar duplicados basados en 'id', conservando la primera aparición
negocios_limpio = negocios_limpio.drop_duplicates(subset=['id'], keep='first')

In [23]:
# Buscar duplicados en 'city' (sin eliminar nada)
duplicados = ciudades[ciudades.duplicated(subset=['city'], keep=False)]

# Mostrar los duplicados en consola
print("Pares duplicados de ciudades:")
print(duplicados.sort_values(by='city'))


Pares duplicados de ciudades:
     city_id            city  population
333      336  Pine Ridge Cdp       10676
670      692  Pine Ridge Cdp        1800
83        84  University Cdp       48868
97        98  University Cdp       42481
483      496   Whitfield Cdp        4745
792      834   Whitfield Cdp         626


In [24]:
# Eliminar duplicados basados en la columna 'city' (mantiene la primera ocurrencia)
ciudades = ciudades.drop_duplicates(subset=['city']) 

#Encontrar los `city` que tenían duplicados
duplicados = ["Pine Ridge Cdp", "University Cdp", "Whitfield Cdp"]

# Ver qué `city_id` quedaron después de eliminar duplicados
ciudades_filtradas = ciudades[ciudades['city'].isin(duplicados)]

print(ciudades_filtradas)

     city_id            city  population
83        84  University Cdp       48868
333      336  Pine Ridge Cdp       10676
483      496   Whitfield Cdp        4745


In [25]:
# Diccionario con los `city_id` a reemplazar
mapeo_city_id = {
    692: 336,  # Pine Ridge Cdp
    98: 84,    # University Cdp
    834: 496   # Whitfield Cdp
}

# Reemplazar city_id en el DataFrame de negocios
negocios_limpio['city_id'] = negocios_limpio['city_id'].replace(mapeo_city_id)

## Revisión de reviews y unificacion

In [26]:
reviews_gm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271477 entries, 0 to 271476
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              271477 non-null  object 
 1   rating          271477 non-null  int64  
 2   has_text        271477 non-null  int64  
 3   vader_score     271477 non-null  float64
 4   textblob_score  271477 non-null  float64
 5   date            271477 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 12.4+ MB


In [27]:
# Convertir tipos de datos
reviews_gm['id'] = reviews_gm['id'].astype(str)
reviews_gm['rating'] = reviews_gm['rating'].astype('int64')
reviews_gm['has_text'] = reviews_gm['has_text'].astype('int64')

# Redondear los puntajes a 3 decimales
reviews_gm['vader_score'] = reviews_gm['vader_score'].round(3)
reviews_gm['textblob_score'] = reviews_gm['textblob_score'].round(3)

# Convertir la columna 'date' a formato datetime
reviews_gm['date'] = pd.to_datetime(reviews_gm['date'])

# Verificar la estructura después de la conversión
print(reviews_gm.info())
print(reviews_gm.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271477 entries, 0 to 271476
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              271477 non-null  object        
 1   rating          271477 non-null  int64         
 2   has_text        271477 non-null  int64         
 3   vader_score     271477 non-null  float64       
 4   textblob_score  271477 non-null  float64       
 5   date            271477 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 12.4+ MB
None
                         id  rating  has_text  vader_score  textblob_score  \
0  gormleysonth297269849815       5         1        0.958           0.206   
1  gormleysonth297269849815       1         1        0.919           0.497   
2  gormleysonth297269849815       5         1        0.681           0.100   
3  gormleysonth297269849815       5         1        0.791         

In [28]:
# Convertir tipos de datos
reviews_y['id'] = reviews_y['id'].astype(str)
reviews_y['rating'] = reviews_y['rating'].astype('int64')
reviews_y['has_text'] = reviews_y['has_text'].astype('int64')

# Redondear los puntajes a 3 decimales
reviews_y['vader_score'] = reviews_y['vader_score'].round(3)
reviews_y['textblob_score'] = reviews_y['textblob_score'].round(3)

# Convertir la columna 'date' a formato datetime
reviews_y['date'] = pd.to_datetime(reviews_y['date'])

# Verificar la estructura después de la conversión
print(reviews_y.info())
print(reviews_y.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480440 entries, 0 to 480439
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              480440 non-null  object        
 1   rating          480440 non-null  int64         
 2   has_text        480440 non-null  int64         
 3   vader_score     480440 non-null  float64       
 4   textblob_score  480440 non-null  float64       
 5   date            480440 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 22.0+ MB
None
                         id  rating  has_text  vader_score  textblob_score  \
0  avaxxxxxxxxx279362824828       1         1       -0.459          -0.078   
1  deccanspicex278944826674       5         1        0.971           0.367   
2  daddydawgsho280233827082       5         1        0.637           0.417   
3  lobsterhaven280300826384       5         1        0.988         

In [29]:
# Asegurar que ambas tablas tienen las mismas columnas y en el mismo orden
columnas_ordenadas = ['id', 'rating', 'has_text', 'vader_score', 'textblob_score', 'date']

reviews_gm = reviews_gm[columnas_ordenadas]
reviews_y = reviews_y[columnas_ordenadas]

# Concatenar las tablas
reviews = pd.concat([reviews_gm, reviews_y], ignore_index=True)

# Verificar la estructura después de la concatenación
print(reviews.info())
print(reviews.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751917 entries, 0 to 751916
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              751917 non-null  object        
 1   rating          751917 non-null  int64         
 2   has_text        751917 non-null  int64         
 3   vader_score     751917 non-null  float64       
 4   textblob_score  751917 non-null  float64       
 5   date            751917 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 34.4+ MB
None
                         id  rating  has_text  vader_score  textblob_score  \
0  gormleysonth297269849815       5         1        0.958           0.206   
1  gormleysonth297269849815       1         1        0.919           0.497   
2  gormleysonth297269849815       5         1        0.681           0.100   
3  gormleysonth297269849815       5         1        0.791         

In [30]:
# Limpiar la columna 'id': quitar espacios y acentos
reviews['id'] = reviews['id'].apply(lambda x: unidecode(x).replace(" ", ""))

In [31]:
# Guardar las tablas
negocios_limpio.to_csv(r'modelado/negocios.csv', index = False)
ciudades.to_csv(r'modelado/ciudades.csv', index = False)
reviews.to_csv(r'modelado/reviews.csv', index = False)
categorias.to_csv(r'modelado/categorias.csv', index = False)

In [35]:
print(negocios_categorias.info())
print(negocios_categorias.head())
print(categorias.info())
print(categorias.head())

<class 'pandas.core.frame.DataFrame'>
Index: 36326 entries, 0 to 36521
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           36326 non-null  object
 1   category_id  36326 non-null  object
dtypes: object(2)
memory usage: 851.4+ KB
None
                         id category_id
0  aamorepizzer278603827290          26
1  aapizzaxxxxx283578826647          19
2  aapizzaxxxxx283578826647          26
3  aapizzaxxxxx283578826647          27
4  abbottstatio282351821827          37
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  66 non-null     int64 
 1   category     65 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.2+ KB
None
   category_id    category
0            1     Chinese
1            2    Japanese
2            3      Korean
3            4   

In [ ]:
# Unir los DataFrames en base a 'category_id'
negocios_con_categorias = negocios_categorias.merge(categorias, on='category_id', how='left')

# Seleccionar solo las columnas 'id' y 'category'
negocios_con_categorias = negocios_con_categorias[['id', 'category']]

# Opcional: Reemplazar valores nulos en 'category' si es necesario
negocios_con_categorias.fillna('None', inplace=True)

# Mostrar los primeros resultados
print(negocios_con_categorias.head())

                         id             category
0  aamorepizzer278603827290                Pizza
1  aapizzaxxxxx283578826647     Western European
2  aapizzaxxxxx283578826647                Pizza
3  aapizzaxxxxx283578826647  Burgers & Fast Food
4  abbottstatio282351821827           Restaurant


In [38]:
# Unir los DataFrames en base a 'atributo_id'
negocios_con_atributos = negocios_atributos.merge(df_atributos, on='atributo_id', how='left')

# Seleccionar solo las columnas 'id' y 'category'
negocios_con_atributos = negocios_con_atributos[['id', 'atributo']]

# Opcional: Reemplazar valores nulos en 'category' si es necesario
# negocios_con_atributos.fillna('None', inplace=True)

# Mostrar los primeros resultados
print(negocios_con_atributos.head())

                         id             atributo
0  aamorepizzer278603827290             delivery
1  aamorepizzer278603827290              takeout
2  aamorepizzer278603827290      outdoor_seating
3  aamorepizzer278603827290  wheelchair_friendly
4  aamorepizzer278603827290     alcohol_beverage


In [40]:
negocios_con_atributos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249747 entries, 0 to 249746
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        249747 non-null  object
 1   atributo  249747 non-null  object
dtypes: object(2)
memory usage: 3.8+ MB


In [41]:
negocios_con_categorias.to_csv(r'modelado/categorias_sin_id.csv', index=False)
negocios_con_atributos.to_csv(r'modelado/atributos_sin_id.csv', index=False)